<a href="https://colab.research.google.com/github/LimYeongJun/Jump-to-python/blob/master/ch08_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install tensorflow-gpu==2.0.0

In [0]:
import tensorflow as tf
print(tf.__version__)

2.0.0


# ch.08. 이미지를 분류합니다 - **합성곱 신경망**

7장에서는 다중 분류 신경망을 만들었습니다. 하지만 패션 MNIST 데이터를 분류하는 데 이 신경망으로 학습을 했더니 정확도가 약 86% 가 나왔습니다. 이는 오차가 적은 편이긴 하지만 실제로 적용을 하기에는 조금 무리가 있습니다. 따라서 이미지 분류에 특화된 **합성곱 신경망** 과 합성곱 신경망을 위한 여러가지 개념들을 도입하므로써 정확도를 높일 수 있습니다. 합성곱 신경망은 완전연결 신경망에 비해 훨씬 가중치가 적으면서, 이미지 분류 문제를 더 잘 해결할 수 있기 때문에 널리 이용되는 방법입니다.

- CNN (Convolutional Neural Network) --> 합성곱 신경망<br>
CNN은 이미지 인식과 음성인식 등 다양한 곳에서 사용되는데, 특히 이미지 인식 분야에서 딥러닝을 활용한 기법은 거의 CNN 을 기초로 합니다.

- 완전연결 계층의 문제점을 보완한 것이 CNN <br>
이미지 데이터는 보통 3차원 (가로, 세로, 채널) 으로 구성되어 있지만, 완전연결 계층에서는 이 형상을 무시하고 1차원 데이터로 간주하여 처리를 합니다. 그래서 정확성이 떨어집니다. 예를 들어, 공간적으로 가까운 픽셀은 값이 비슷하거나, 각 채널은 서로 밀접하게 연관이 되어 있습니다. 이를 1차원 데이터로 나열하게 되면 정확성이 떨어질 수 밖에 없고, 이를 보완한 것이 CNN 방법입니다. CNN은 이미지를 3차원 데이터로 입력을 받을 수 있으며, 마찬가지로 다음 계층에도 3차원 데이터로 전달할 수 있기 때문에 완전 연결의 문제점을 보완하여 정확성을 높일 수 있습니다.

<img src="https://github.com/LimYeongJun/LimYeongJun/blob/master/KakaoTalk_20191120_004034399.jpg?raw=true" width="100%">

<img src="https://github.com/LimYeongJun/LimYeongJun/blob/master/KakaoTalk_20191120_004034399_01.jpg?raw=true" width="100%">

**affine 계층 : 완전 연결 계층**



## 08-1 합성곱 연산

합성곱 : 두 함수에 적용하여 새로운 함수를 만드는 수학 연산자

두 배열 x , w 가 있을 때 원소수가 적은 배열 w의 원소 순서를 뒤집어보면
<img src="https://github.com/LimYeongJun/LimYeongJun/blob/master/KakaoTalk_20191117_170620463.jpg?raw=true" width="60%">

뒤집은 배열을 배열 x의 왼쪽 끝자리에 맞춰 놓고, 각 배열 원소끼리 곱한 후 더합니다.

<img src="https://github.com/LimYeongJun/LimYeongJun/blob/master/KakaoTalk_20191117_170620463_11.jpg?raw=true" width="80%">

그 후 $w^r$ 을 오른쪽으로 한 칸 이동하여 각 배열 원소끼리 곱한 후 더합니다.

<img src="https://github.com/LimYeongJun/LimYeongJun/blob/master/KakaoTalk_20191117_170620463_21.jpg?raw=true" width="80%">

이러한 규칙으로 계속 계산을 해 나가면 다음과 같은 결과가 출력됩니다.

<img src="https://github.com/LimYeongJun/LimYeongJun/blob/master/KakaoTalk_20191117_170620463_22.jpg?raw=true" width="60%">

지금까지 합성곱의 개념이었고, 이를 넘파이로 구현해보겠습니다.






In [0]:
import numpy as np
w = np.array([2, 1, 5, 3])
x = np.array([2, 8, 3, 7, 1, 2, 0, 4, 5])

w_r = np.flip(w)
print(w_r)

[3 5 1 2]


In [0]:
for i in range(6):
  print(np.dot(x[i:i+4], w_r))

63
48
49
28
21
20


scipy로 구현해보면

In [0]:
from scipy.signal import convolve
convolve(x, w, mode='valid') 

array([63, 48, 49, 28, 21, 20])

교차상관 : 합성곱 연산에서 원소수가 적은 배열 w의 원소 순서를 뒤집지 않음<br>
<img src="https://github.com/LimYeongJun/LimYeongJun/blob/master/KakaoTalk_20191117_170620463_23.jpg?raw=true" width="50%">

이 또한 scipy의 correlate( )함수를 이용해서 계산할 수 있습니다.

In [0]:
from scipy.signal import correlate
correlate(x, w, mode = 'valid')

array([48, 57, 24, 25, 16, 39])

대부분의 딥러닝 패키지들은 합성곱 연산을 사용하지 않고 교차상관을 쓰지만, 관례적으로 합성곱을 쓴다고 말합니다. 왜냐하면, 모든 모델은 훈련하기 전에 가중치 배열의 요소들을 무작위로 초기화 하기 때문에, 가중치를 뒤집어서 사용하던지, 그대로 사용하던지 상관이 없기 때문입니다.

## 패딩과 스트라이드

패딩(padding) : 원본 배열의 양 끝에 빈 원소를 추가하는 것 <br>
스트라이드(stride) : 미끄러지는 배열의 간격을 조절하는 것

먼저 패딩을 살펴보면<br>
패딩 (padding)은 3가지로 구분.
- valid padding
- full padding
- same padding


<img src="https://github.com/LimYeongJun/LimYeongJun/blob/master/KakaoTalk_20191117_170620463_24.jpg?raw=true" width="70%">

# **< valid padding>** : 원본 배열의 원소가 합성곱 연산에 참여하는 정도가 다름

- 이로 인해 밸리드 패딩의 결과로 얻는 배열의 크기는 원본 배열보다 항상 작음.
이를 해결하기 위해 풀 패딩 이용

<img src="https://github.com/LimYeongJun/LimYeongJun/blob/master/KakaoTalk_20191117_170620463_25.jpg?raw=true" width="70%">

# **< full padding>** : 원본 배열의 원소가 연산에 동일하게 참여

<img src="https://github.com/LimYeongJun/LimYeongJun/blob/master/KakaoTalk_20191117_170620463_26.jpg?raw=true" width="90%">

# **< same padding>** : 출력 배열의 길이를 원본 배열의 길이와 동일하게 만듬.

주로 same padding 사용. 왜냐하면, 패딩은 주로 출력 크기를 조정할 목적으로 사용하기 때문입니다. 예를 들어, (4,4) 입력 데이터에 (3,3) 가중치 필터를 적용하면 stride값을 1로 설정했을 때 출력은 (2,2)가 되어 입력보다 2만큼 줄어들게 됩니다. 이는 합성곱 연산을 되풀이하는 신경망에서는 문제가 될 수 있기 때문입니다. 계속 연산을 하다 보면 어느 순간 출력의 크기가 1로 되어버리기 때문에 합성곱 연산을 적용할 수 없습니다. 따라서 입력 데이터의 공간적 크기를 고정한 채로 다음 계층에 전달할 수 있습니다.





그 다음 스트라이드에 대해 알아보겠습니다.

스트라이드는 미끄러지는 간격을 조정합니다.

<img src="https://github.com/LimYeongJun/LimYeongJun/blob/master/KakaoTalk_20191117_170620463_27.jpg?raw=true" width="70%">

지금까지 사용한 패딩(valid padding, full padding, same padding)은 스트라이드를 1로 지정하여 연산을 수행했었습니다. 만약 스트라이드를 2로 지정한다면 위와 같은 그림이 됩니다.<br>

이와 같이 패딩과 스트라이드를 가지고 2차원 배열에서 합성곱을 수행할 수 있습니다.

## **2차원 배열에서 합성곱 연산 수행**

<img src="https://github.com/LimYeongJun/LimYeongJun/blob/master/KakaoTalk_20191117_170620463_01.jpg?raw=true" width="50%">

<img src="https://github.com/LimYeongJun/LimYeongJun/blob/master/KakaoTalk_20191117_170620463_02.jpg?raw=true" width="50%">

<img src="https://github.com/LimYeongJun/LimYeongJun/blob/master/KakaoTalk_20191117_170620463_03.jpg?raw=true" width="1000%">






**scipy**의 correlate2d( ) 내장함수를 이용하여 2차원 배열의 합성곱을 계산할 수 있습니다.

In [0]:
x = np.array([[1,2,3],[4,5,6],[7,8,9]])
w = np.array([[2,0], [0,0]])
from scipy.signal import correlate2d
correlate2d(x, w, mode = 'valid')

array([[ 2,  4],
       [ 8, 10]])

2차원 배열에서 padding 과 stride 도 한번 보면

<img src="https://github.com/LimYeongJun/LimYeongJun/blob/master/KakaoTalk_20191117_170620463_04.jpg?raw=true" width="100%">

위의 그림과 같이 되는데, 여기서 원본 배열의 크기와 같은 출력 배열을 만들기 위해 오른쪽과 아래쪽 모서리에 제로패딩을 추가했습니다. 

In [0]:
correlate2d(x, w, mode = 'same')

array([[ 2,  4,  6],
       [ 8, 10, 12],
       [14, 16, 18]])

stride는 미끄러지는 방향을 그대로 유지하면서, 미끄러지는 간격의 크기만 커지므로 아래 그림과 같이 나타낼 수 있습니다.

<img src="https://github.com/LimYeongJun/LimYeongJun/blob/master/KakaoTalk_20191117_170620463_05.jpg?raw=true" width="1000%">

지금까지는 scipy를 이용한 합성곱을 수행해봤는데, 텐서플로를 이용한 연산도 가능합니다. 합성곱 신경망의 입력은 일반적으로 4차원 배열입니다. 이는 입력 이미지의 높이와 너비 외에 더 많은 차원이 필요하기 때문인데, 구체적으로 입력 배열이 어떻게 구성되는 지 알아보겠습니다.

<img src="https://github.com/LimYeongJun/LimYeongJun/blob/master/KakaoTalk_20191117_170620463_06.jpg?raw=true" width="100%">

- 입력에 2개의 샘플이 포함
- 각 샘플은 R, G, B 로 구분되는 3개의 컬러 채널을 가지고 있음.
- 위 그림에서 입력을 4차원 배열로 표현하면 (배치, 샘플의 높이, 샘플의 너비, 컬러 채널의 차원) 으로 표현되고, (2, 3, 3, 3) 으로 표현가능함.


<img src="https://github.com/LimYeongJun/LimYeongJun/blob/master/KakaoTalk_20191117_170620463_07.jpg?raw=true" width="100%">

- 입력과 곱해지는 가중치도 똑같이 4개의 차원으로 , (가중치의 높이, 가중치의 너비, 채널, 가중치의 개수) 로 표현
- 여기서는 (2, 2, 3, 3) 으로 표현가능함.

따라서 입력과 가중치에 세임 패딩을 적용하여 합성곱을 수행하면 (입력의 배치, 입력의 높이, 입력의 너비, 가중치의 갯수) 가 됩니다.

전체적인 연산 과정은 다음 그림과 같이 나타납니다.

<img src="https://github.com/LimYeongJun/LimYeongJun/blob/master/KakaoTalk_20191117_170620463_08.jpg?raw=true" width="60%">

합성곱의 개념을 알아보았고, 이제 같은 내용을 텐서플로를 이용하여 연산 해 보겠습니다.

In [0]:
import tensorflow as tf

In [0]:
x_4d = x.astype(np.float).reshape(1, 3, 3, 1) #넘파이의 reshape()메서드 이용   #자료를 실수형으로 바꿈
w_4d = w.reshape(2, 2, 1, 1)

In [0]:
c_out = tf.nn.conv2d(x_4d, w_4d, strides=1, padding='SAME')  
c_out.numpy().reshape(3, 3)  # colab에서 tensorflow의 버전이 1.14로 설정되어있어서 자꾸 오류가 떴음. 2.0 버전으로 바꾼 후 오류가 뜨지 않음.

array([[ 2.,  4.,  6.],
       [ 8., 10., 12.],
       [14., 16., 18.]])

잠시 7장의 내용을 간단하게 상기해보면, 패션 MNIST 데이터 세트를 MultiClassNetwork 클래스에 적용하며, 28x28 크기의 입력을 일렬로 펼쳤습니다. 따라서 가중치의 개수도 78,400 개로 매우 많이 필요했습니다. 하지만 합성곱 신경망에서는 28x28 크기의 입력을 펼치지 않고 그대로 사용하여 3x3 또는 5x5 크기의 가중치로 합성곱을 적용하여 가중치 배열의 크기는 훨씬 작아졌고 입력의 특징을 더 잘 찾기 때
문에 합성곱 신경망이 이미지 분류에서 뛰어난 성능을 발휘할 수 있습니다.

# 08-2 풀링 연산에 대해 알아봅니다.

합성곱이 일어나는 층을 합성곱층, 풀링이 일어나는 층을 풀링층이라고 부릅니다.<br>
합성곱층과 풀링층에서 만들어진 결과를 특성맵(feature map) 이라고 부릅니다.

<img src="https://github.com/LimYeongJun/LimYeongJun/blob/master/KakaoTalk_20191117_170620463_09.jpg?raw=true" width="80%">

- 입력이 합성곱층을 통과할 때 합성곱과 활성화 함수가 적용되어 특성 맵이 만들어짐.
- 그 후 특성 맵이 풀링층을 통과하여 또 다른 특성 맵이 만들어짐.
- 풀링 : 특성 맵을 스캔하며 최댓값을 고르거나 평균값을 계산하는 것을 말함.

<img src="https://github.com/LimYeongJun/LimYeongJun/blob/master/KakaoTalk_20191117_170620463_10.jpg?raw=true" width="80%">

# **<최대 풀링>**

- 최대 풀링은 특성 맵의 크기를 절반으로 줄임.
- 합성곱 층에서 스트라이드를 크게 지정하여 특성 맵의 크기를 줄여도 되지만, 세임패딩을 적용 후 풀링층에서 특성 맵의 크기를 줄이는 것이 더 효율적임.

<img src="https://github.com/LimYeongJun/LimYeongJun/blob/master/KakaoTalk_20191117_170620463_13.jpg?raw=true" width="90%">

# **<평균 풀링>**

보통 평균 풀링보다는 최대 풀링을 더 선호합니다. 그 이유는<br>
- 입력에서 합성곱 필터가 찾고자 하는 부분은 특성 맵의 가장 큰 값으로 활성화되는데 평균 풀링은 가장 큰 특성의 값을 상쇄시키기 때문에 적용하기엔 좀 무리가 있음.
- 최대풀링은 가장 큰 특징을 유지시키는 성질이 있기 때문에 이미지 분류에 잘 맞음.

이와 같은 풀링을 코드로 시행해보면

In [0]:
x = np.array([[1,2,3,4],[5,6,7,8],[9,10,11,12],[13,14,15,16]])
x = x.reshape(1,4,4,1)

In [0]:
p_out = tf.nn.max_pool2d(x, ksize=2, strides=2, padding='VALID')
p_out.numpy().reshape(2,2)

array([[ 6,  8],
       [14, 16]])

이 결과로 보아 성공적으로 최대 풀링 연산이 출력되었습니다.

풀링에서 가장 중요한 점은
- 풀링층에는 학습되는 가중치가 없음.
- 배치 차원이나 채널 차원으로 적용되지 않음. 즉, 풀링층을 통과하기 전후로 배치 크기와 채널 크기는 동일함.

# 08-3 합성곱 신경망의 구조를 알아봅니다.

활성화 함수는 여러 가지 있지만 대표적인 2개를 살펴보면

1. sigmoid 활성화 함수

신경망에서 자주 이용하는 활성화함수로써 $h(x) = \frac{1}{1 + {e}^{-x}}$ 로 나타낼 수 있습니다.<br>
신경망에서는 활성화 함수로 시그모이드 함수를 이용하여 신호를 변환하고, 그 변환된 신호를 다음 뉴런에 전달합니다. 

2. Relu 활성화 함수

Relu 활성화 함수 역시 신경망에서 자주 이용합니다. 수식으로는 $y=\left\{\begin{matrix}
x(x>0)\\ 
0(x\leq 0)
\end{matrix}\right.$ 으로 나타낼 수 있습니다.

<img src="https://github.com/LimYeongJun/LimYeongJun/blob/master/KakaoTalk_20191117_170620463_14.jpg?raw=true" width="90%">

출력층이 이진 분류일 경우에는 sigmoid 함수를 사용했었고(4장), 다중 분류일 경우 softmax 함수를 사용했습니다.(7장) 여기서는 합성곱층에 적용되는 활성화 함수로써 ReLU 활성화 함수를 사용합니다.

이를 python 코드로 구현해 보면






In [0]:
def relu(x):
  return np.maximum(x,0)

In [0]:
x = np.array([-1,2,-3,4,-5])
relu(x)

array([0, 2, 0, 4, 0])

텐서플로가 제공하는 렐루함수는 relu() 이고, 화면에 출력하려면 넘파이로 변환해야 합니다.

In [0]:
r_out = tf.nn.relu(x)
r_out.numpy()

array([0, 2, 0, 4, 0])

이런식으로 똑같은 결과를 얻을 수 있습니다.<br>
ReLU 함수의 도함수는 $y=\left\{\begin{matrix}
1(x>0)\\ 
0(x\leq 0)
\end{matrix}\right.$ 으로 나타낼 수 있습니다.

## 합성곱 신경망에서 일어나는 일들과 구조를 알아봅니다.

- 합성곱 신경망은 이미지의 2차원 형태를 입력으로 그대로 사용하므로 이미지를 한 줄로 펼칠 필요가 없음
- 이러한 특성 때문에 이미지 정보가 손상되지 않음.

<img src="https://github.com/LimYeongJun/LimYeongJun/blob/master/KakaoTalk_20191117_170620463_15.jpg?raw=true" width="40%">

위의 그림과 같이 이미지는 채널이라는 차원을 가지고, 이 채널은 이미지의 픽셀이 가진 색상을 표현하기 위해 필요한 정보.

<img src="https://github.com/LimYeongJun/LimYeongJun/blob/master/KakaoTalk_20191117_170620463_16.jpg?raw=true" width="90%">

<img src="https://github.com/LimYeongJun/LimYeongJun/blob/master/KakaoTalk_20191117_170620463_17.jpg?raw=true" width="90%">

<img src="https://github.com/LimYeongJun/LimYeongJun/blob/master/KakaoTalk_20191117_170620463_18.jpg?raw=true" width="90%">

<img src="https://github.com/LimYeongJun/LimYeongJun/blob/master/KakaoTalk_20191117_170620463_19.jpg?raw=true" width="90%">

